In [ ]:
from urllib.request import urlopen
import json
import numpy as np
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import plotly.graph_objects as go
import plotly.express as px

# Визуализация данных на российской карте библиотекой Plotly

## Собираем данные 

### стопкоронавирус.рф

In [ ]:
driver = webdriver.Chrome()
driver.get('https://стопкоронавирус.рф/information/')
source_data = driver.page_source
soup = bs(source_data, 'lxml')

Создаем список регионов **region_names_list** (наименования регионов находятся под тегом \<th>)

In [ ]:
divs_region_names = soup.find_all('th', {'class':'col-region'})
region_list_covid = []
for i in range(1, len(divs_region_names)):
    region_name = divs_region_names[i].text
    region_name = region_name.replace('\n', '').replace('  ', '').strip()
    region_list_covid.append(region_name)

#### Создаем списки с данными (данные находятся с столбцах таблицы под тегом \<td>)

In [ ]:
divs_data = soup.find_all('td')
sick_list = []    # выявлено
new_list = []     # новые
healed_list = []  # выздоровело
died_list = []    # умерло

In [ ]:
divs_data[2:6]

In [ ]:
counter = 1
for td in divs_data[2:]:
    if counter == 1:
        sick_list.append(int(td.text))
    elif counter == 2:
        new_list.append(int(td.text))
    elif counter == 3:
        healed_list.append(int(td.text))
    elif counter == 4:
        died_list.append(int(td.text))
        counter = 0
    counter += 1

#### Объединяем списки в датафрейм **df_covid**

In [ ]:
df_covid = pd.DataFrame()
df_covid['region_name'] = region_list_covid
df_covid['sick'] = sick_list
df_covid['new'] = new_list
df_covid['healed'] = healed_list
df_covid['died'] = died_list

In [ ]:
df_covid.head()

### geojson

Скачаем открытый geojson с границами российских регионов, найденный по одной из первых ссылок в Google по запросу «russia geojson». В нём уже есть кое-какие данные: например, наименования регионов. Но этот geojson-файл пока ещё не подходит под требуемый формат Plotly: в нём не размечены идентификаторы регионов.

In [ ]:
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/russia.geojson') as response:
    counties = json.load(response)

#### Создаем список с регионами из counties и сравниваем со списком регионов из df_covid

In [ ]:
region_list_geojson = [counties['features'][i]['properties']['name'] for i in range(len(counties['features']))]

In [ ]:
set_regions_covid = set(df_covid.region_name.to_list())
set_region_geojson = set(region_list_geojson)
diff = set_regions_covid.symmetric_difference(set_region_geojson)
diff

Кроме разметки идентификаторов есть различия в наименовании регионов. Например, на сайте **стопкоронавирус.рф**, откуда мы будем брать свежие данные о заболевших, республика Башкортостан занесена как «Республика Башкортостан», а в нашем geojson-файле — просто «Башкортостан». Все эти различия необходимо устранить во избежание конфликтов. Кроме того, все первые буквы в названиях регионов должны начинаться с верхнего регистра.

Вносим правки в **counties**:

In [ ]:
for k in range(len(counties['features'])):
    counties['features'][k]['id'] = k
    if counties['features'][k]['properties']['name'] == 'Бурятия':
        counties['features'][k]['properties']['name'] = 'Республика Бурятия'
    elif counties['features'][k]['properties']['name'] == 'Тыва':
        counties['features'][k]['properties']['name'] = 'Республика Тыва'
    elif counties['features'][k]['properties']['name'] == 'Ханты-Мансийский автономный округ - Югра':
        counties['features'][k]['properties']['name'] = 'Ханты-Мансийский АО'
    elif counties['features'][k]['properties']['name'] == 'Адыгея':
        counties['features'][k]['properties']['name'] = 'Республика Адыгея'
    elif counties['features'][k]['properties']['name'] == 'Татарстан':
        counties['features'][k]['properties']['name'] = 'Республика Татарстан'
    elif counties['features'][k]['properties']['name'] == 'Марий Эл':
        counties['features'][k]['properties']['name'] = 'Республика Марий Эл'
    elif counties['features'][k]['properties']['name'] == 'Чувашия':
        counties['features'][k]['properties']['name'] = 'Республика Чувашия'
    elif counties['features'][k]['properties']['name'] == 'Северная Осетия - Алания':
        counties['features'][k]['properties']['name'] = 'Республика Северная Осетия — Алания' 
    elif counties['features'][k]['properties']['name'] == 'Алтай':
        counties['features'][k]['properties']['name'] = 'Республика Алтай'
    elif counties['features'][k]['properties']['name'] == 'Дагестан':
        counties['features'][k]['properties']['name'] = 'Республика Дагестан'
    elif counties['features'][k]['properties']['name'] == 'Ингушетия':
        counties['features'][k]['properties']['name'] = 'Республика Ингушетия'
    elif counties['features'][k]['properties']['name'] == 'Башкортостан':
        counties['features'][k]['properties']['name'] = 'Республика Башкортостан'
    elif counties['features'][k]['properties']['name'] in ['Удмуртская республика',
                                                           'Кабардино-Балкарская республика',
                                                           'Карачаево-Черкесская республика',
                                                           'Чеченская республика']:
        counties['features'][k]['properties']['name'] = counties['features'][k]['properties']['name'].title()

Делаем повторное сравнение:

In [ ]:
region_list_geojson = [counties['features'][i]['properties']['name'] for i in range(len(counties['features']))]

In [ ]:
set_regions_covid = set(df_covid.region_name.to_list())
set_region_geojson = set(region_list_geojson)
diff = set_regions_covid.symmetric_difference(set_region_geojson)
diff

**Так лучше!**

Из получившегося geojson-файла сформируем DataFrame с регионами России: возьмём идентификаторы и наименования.

In [ ]:
region_id_list = []
regions_list = []
for k in range(len(counties['features'])):
    region_id_list.append(counties['features'][k]['id'])
    regions_list.append(counties['features'][k]['properties']['name'])
df_geojson = pd.DataFrame()
df_geojson['region_id'] = region_id_list
df_geojson['region_name'] = regions_list

In [ ]:
df_geojson.head()

### wikipedia.org

In [ ]:
driver = webdriver.Chrome()
driver.get('https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8')
source_data = driver.page_source
soup_wiki = bs(source_data, 'lxml')

#### Создаем списки с данными (данные находятся с столбцах таблицы под тегом \<td>)

In [ ]:
divs_data_wiki = soup_wiki.find_all('td')

In [ ]:
divs_data_wiki[1:10]

In [ ]:
region_name = []
popul = []
percent = []
city_pop_percent = []
square = []
density = []

counter = 1
for td in divs_data_wiki[1:]:
    if td.text.strip() == 'РФ':
        break
    if counter == 1:
        region_name.append(td.text.strip())
    elif counter == 2:
        population = ''
        tmp_pop = td.text
        for symbol in tmp_pop:
            if symbol.isdigit():
                population += symbol
        popul.append(int(population))
    elif counter == 3:
        percent.append(float(td.text.strip().replace(',', '.')))
    elif counter == 5:
        city_pop_percent.append(float(td.text.strip().replace(',', '.')))
    elif counter == 8:
        sq = ''
        tmp_square = td.text
        for symbol in tmp_square:
            if symbol.isdigit():
                sq += symbol
        square.append(int(sq))
    elif counter == 9:
        density.append(float(td.text.strip().replace(',', '.')))
    if counter == 11:
        counter = 0
    counter += 1

In [ ]:
divs_data_wiki[8].text.strip()

In [ ]:
sorted(region_name)[:10]

#### Сравниваем списки регионов region_name (с сайта wikipedia) со списком из df_covid

In [ ]:
set_region_wiki = set(region_name)
set_regions_covid.symmetric_difference(set_region_wiki)

Вносим правки в **region_name** (с сайта wikipedia):

In [ ]:
for r in enumerate(region_name):
    if r[1] == 'Архангельская область (с НАО)':
        region_name[r[0]] = 'Архангельская область'
    elif r[1] == 'Республика СевернаяОсетия — Алания':
        region_name[r[0]] = 'Республика Северная Осетия — Алания'
    elif r[1] == 'Чувашская Республика':
        region_name[r[0]] = 'Республика Чувашия'
    elif r[1] == 'Тюменская областьбез ХМАО и ЯНАО':
        region_name[r[0]] = 'Тюменская область'
    elif r[1] == 'Ханты-Мансийскийавтономный округ — Югра':
        region_name[r[0]] = 'Ханты-Мансийский АО'

Делаем повторное сравнение

In [ ]:
set_region_wiki = set(region_name)
set_regions_covid.symmetric_difference(set_region_wiki)

Создаем датафрейм:

In [ ]:
df_wiki = pd.DataFrame({'region_name': region_name,
                        'population': popul,
                        'percent': percent,
                        'city_pop_percent': city_pop_percent,
                        'square': square,
                        'density': density
                       })
df_wiki.head()

## Объединяем собранные данные

In [ ]:
# df = pd.DataFrame()

In [ ]:
df = df_wiki.merge(df_geojson, on='region_name')
df = df.merge(df_covid, on='region_name')

In [ ]:
df.head()

Объединяем датафреймы и добавляем колонки:
* **sick_percent_full** - процент людей в регионе с выявленным COVID-19
* **sick_per_day** - количество заболевших людей в регионе на миллион человек за день
* **died_per_mln** - количество умерших людей в регионе на миллион человек

In [ ]:
df['sick_percent_full'] = round(100 * df.sick / df.population, 2)
df['sick_per_day'] = round(1_000_000 * df.new / df.population, 2)
df['died_per_mln'] = round(1_000_000 * df.died / df.population, 2)

In [ ]:
df.head()

In [ ]:
df.shape

### Визуализация данных на карте Plotly

#### Процент людей в регионе с выявленным COVID-19

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties,
                           locations=df['region_id'],
                           z=df['sick_percent_full'],
                           text=df['region_name'],
                           colorscale='Reds',
                           colorbar_thickness=20,
                           customdata=np.stack([df['new'], df['sick'] - df['healed'], df['died'], df['sick'], df['healed']], axis=-1),
                           hovertemplate='<b>%{text}</b>'+ '<br>' +
                                         'Процент выявленных случаев: %{z}' + '<br>' +
                                         'Новых: %{customdata[0]}' + '<br>' +
                                         'Активных: %{customdata[1]}' + '<br>' +
                                         'Умерло: %{customdata[2]}' + '<br>' +
                                         'Всего случаев: %{customdata[3]}' + '<br>' +
                                         'Выздоровело: %{customdata[4]}' +
                                         '<extra></extra>',
                           hoverinfo='text, z'))

In [ ]:
fig.update_layout(title_text = 'Процент людей в регионе с выявленным COVID-19',
                  mapbox_style="carto-positron",
                  mapbox_zoom=2,
                  mapbox_center = {"lat": 66, "lon": 94})
fig.update_traces(marker_line_width=.1)
fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0})
fig.show()

#### Количество умерших людей в регионе на миллион человек

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties,
                           locations=df['region_id'],
                           z=df['died_per_mln'],
                           text=df['region_name'],
                           colorscale='Reds',
                           colorbar_thickness=20,
                           customdata=np.stack([df['new'], df['sick'] - df['healed'], df['died'], df['sick'], df['healed']], axis=-1),
                           hovertemplate='<b>%{text}</b>'+ '<br>' +
                                         'Умерло на миллион человек: %{z}' + '<br>' +
                                         'Новых: %{customdata[0]}' + '<br>' +
                                         'Активных: %{customdata[1]}' + '<br>' +
                                         'Умерло: %{customdata[2]}' + '<br>' +
                                         'Всего случаев: %{customdata[3]}' + '<br>' +
                                         'Выздоровело: %{customdata[4]}' +
                                         '<extra></extra>',
                           hoverinfo='text, z'))

In [ ]:
fig.update_layout(title_text = 'Количество умерших людей в регионе на миллион человек',
                  mapbox_style="carto-positron",
                  mapbox_zoom=2,
                  mapbox_center = {"lat": 66, "lon": 94})
fig.update_traces(marker_line_width=.1)
fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0})
fig.show()

#### Количество заболевших людей в регионе на миллион человек за день

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties,
                           locations=df['region_id'],
                           z=df['sick_per_day'],
                           text=df['region_name'],
                           colorscale='Reds',
                           colorbar_thickness=20,
                           customdata=np.stack([df['new'], df['sick'] - df['healed'], df['died'], df['sick'], df['healed']], axis=-1),
                           hovertemplate='<b>%{text}</b>'+ '<br>' +
                                         'Выявлено за день на миллион человек: %{z}' + '<br>' +
                                         'Новых: %{customdata[0]}' + '<br>' +
                                         'Активных: %{customdata[1]}' + '<br>' +
                                         'Умерло: %{customdata[2]}' + '<br>' +
                                         'Всего случаев: %{customdata[3]}' + '<br>' +
                                         'Выздоровело: %{customdata[4]}' +
                                         '<extra></extra>',
                           hoverinfo='text, z'))

In [ ]:
fig.update_layout(title_text = 'Количество заболевших людей в регионе на миллион человек за день',
                  mapbox_style="carto-positron",
                  mapbox_zoom=2,
                  mapbox_center = {"lat": 66, "lon": 94})
fig.update_traces(marker_line_width=.1)
fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0})
fig.show()

#### Процент городского населения

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties,
                           locations=df['region_id'],
                           z=df['city_pop_percent'],
                           text=df['region_name'],
                           colorscale='Greens',
                           colorbar_thickness=20,
                           customdata=np.stack([df['population']], axis=-1),
                           hovertemplate='<b>%{text}</b>'+ '<br>' +
                                         'Процент городского населения: %{z}' + '<br>' +
                                         'Население региона: %{customdata[0]}' + '<br>' +
                                         '<extra></extra>',
                           hoverinfo='text, z'))

In [ ]:
fig.update_layout(title_text = 'Процент городского населения',
                  mapbox_style="carto-positron",
                  mapbox_zoom=2,
                  mapbox_center = {"lat": 66, "lon": 94})
fig.update_traces(marker_line_width=.1)
fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0})
fig.show()

#### Плотность населения в регионах

In [ ]:
df_dens = df.copy()
df_dens['log_dens'] = np.log(df_dens.density)
df_dens['log_dens'] = df_dens['log_dens'] - min(df_dens['log_dens'])
df_dens.head()

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties,
                           locations=df_dens['region_id'],
                           z=df_dens['log_dens'],
                           text=df_dens['region_name'],
                           colorscale='Blues',
                           colorbar_thickness=20,
                           customdata=np.stack([df_dens['density'], df_dens['population']], axis=-1),
                           hovertemplate='<b>%{text}</b>'+ '<br>' +
                                         'Плотность населения: %{customdata[0]}' + '<br>' +
                                         'Население региона: %{customdata[1]}' + '<br>' +
                                         '<extra></extra>',
                           hoverinfo='text, z'))

In [ ]:
fig.update_layout(title_text = 'Плотность населения в регионах (чел/кв.км)',
                  mapbox_style="white-bg",  # empty white canvas which results in no external HTTP requests
                  mapbox_zoom=1,
                  mapbox_center = {"lat": 66, "lon": 94}
#                   geo_scope="asia"
                 )
fig.update_traces(marker_line_width=.1)
fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0})
fig.show()